In [88]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from string import punctuation
from collections import defaultdict
from functools import reduce
from keras.preprocessing.sequence import pad_sequences
from itertools import chain
from InputPreparator import EmbeddingsPreparator
from InputPreparator import StoryParser
import time
from tqdm import tqdm


In [164]:
train_set_file = get_task_files(10)[0]
test_set_file = get_task_files(10)[1]

train_set_post_file = "tasks_1-20_v1-2/en/"+train_set_file
test_set_post_file = "tasks_1-20_v1-2/en/"+test_set_file

In [165]:
def get_task_files(task_nr):
    if task_nr==5:
        return 'qa5_three-arg-relations_train.txt', "qa5_three-arg-relations_test.txt"
    if task_nr==6:
        return 'qa6_yes-no-questions_train.txt', 'qa6_yes-no-questions_test.txt'
    if task_nr==10:
        return 'qa10_indefinite-knowledge_train.txt', 'qa10_indefinite-knowledge_test.txt'

In [166]:
embedder=EmbeddingsPreparator()
story_parser=StoryParser()

In [167]:
vocab_tokens = embedder.get_unique_tokens([train_set_post_file, test_set_post_file])

In [168]:
word_to_index, index_to_embedding = embedder.load_embedding_from_disks("glove.6B.50d.txt",vocab_tokens, with_indexes=True)

In [182]:
train_stories=story_parser.get_stories(train_set_post_file, True)
test_stories=story_parser.get_stories(test_set_post_file, True)

/Users/sdoneva/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [183]:
contexts_train, questions_train, answers_train = story_parser.vectorize_stories(train_stories, word_to_index)
contexts_test, questions_test, answers_test = story_parser.vectorize_stories(test_stories, word_to_index)

In [184]:
print('contexts.shape = {}'.format(contexts_train.shape))
print('questions.shape = {}'.format(questions_train.shape))
print('answers.shape = {}'.format(answers_train.shape))

contexts.shape = (1000,)
questions.shape = (1000, 6)
answers.shape = (1000, 27)


In [185]:
final_train_data = story_parser.get_final_dataset(contexts_train, questions_train, answers_train)
final_test_data = story_parser.get_final_dataset(contexts_test, questions_test, answers_test)

# Model

In [186]:
tf.reset_default_graph()

### Input

In [187]:
context_ids = tf.placeholder(tf.float32, shape=[None,None,], name='context')  
question_ids = tf.placeholder(tf.float32, shape=[None,None,], name='question')  
true_answer= tf.placeholder(tf.float32, shape=[None,None], name='correct_answer')
dimensions= index_to_embedding.shape[0]

### Layers

In [188]:
embed=tf.keras.layers.Embedding(42,50)
gru=tf.keras.layers.GRUCell(50)
rnn=tf.keras.layers.RNN(gru)
dropout=tf.keras.layers.Dropout(0.5)

### Model Logic

In [189]:
context_embed = embed(context_ids)  
quest_embed = embed(question_ids)

encoded_sentence=rnn(context_embed)
encoded_question=rnn(quest_embed)

encoded_sentence=dropout(encoded_sentence)
encoded_question=dropout(encoded_question)

merged= tf.keras.layers.concatenate([encoded_sentence, encoded_question])

preds = tf.keras.layers.Dense(dimensions, activation='softmax')(merged)  

In [190]:
loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(true_answer, preds))

In [191]:
train_step = tf.train.AdamOptimizer(0.005).minimize(loss)


In [192]:
acc_value = tf.keras.metrics.categorical_accuracy(true_answer, preds)

In [193]:
batch_size=128

In [194]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    
    start_time = time.time()
    for i in tqdm(range(1000)):
        batch = np.random.randint(final_train_data.shape[0], size=batch_size)
        batch_data = final_train_data[batch]
        feed = prep_batch(batch_data)
        
        train_step.run(feed_dict=feed)
    elapsed_time = time.time() - start_time
    print('Training time: ')
    print(elapsed_time)
    
    print('Final Accuracy Score:')
    test_data_feed=prep_batch(final_test_data)
    print (np.mean(acc_value.eval(feed_dict=test_data_feed)))

100%|██████████| 1000/1000 [00:20<00:00, 49.16it/s]


Training time: 
20.34594416618347
Final Accuracy Score:
0.564


In [122]:
def prep_batch(batch_data):
    contextsvs, questionsvs, answers=zip(*batch_data)
    
    #Pad to longest sequence in the batch 
    contexts = list(contextsvs)
    max_context_length = max([len(x) for x in contexts])
    questions = list(questionsvs)
    max_query_length = max(len(x) for x in questionsvs)

    final_contexts=pad_sequences(contextsvs, maxlen=max_context_length) 
    queries=pad_sequences(questionsvs, maxlen=max_query_length)
    
    feed = {context_ids: final_contexts,
                  question_ids: queries,
                  true_answer: answers}
    
    return feed